In [1]:
import pandas as pd
import numpy as np
import os
import missingno as msno
from imblearn.combine import SMOTEENN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('..')
path = os.getcwd()
train_df = pd.read_csv(path+'\\full_impute_df.csv')

In [3]:
train_df = pd.read_csv(path+'\\full_impute_df.csv')

In [4]:
def preprocessing(df, imputer = False):
    #set the index as the id of the loan
    df = df.set_index('SK_ID_CURR')
    #wrong gender code
    df = df.drop(df.loc[df['CODE_GENDER'] == 'XNA'].index)
    #check if binary features are binary
    binary = [i for i in df.columns if 'FLAG' in i]
    for i in binary:
        if len(df[i].unique()) != 2:
            print('Column {} is not binary.'.format(i))
    #extract target
    if df['TARGET'].isna().sum() != 0:
        df = df.drop(df.loc[df['TARGET'].isna()])
    y = df['TARGET']
    df = df.drop(labels = 'TARGET', axis = 1)
    #OHE categorical features
    cat_df = df.select_dtypes(include = ['object'])
    cat_idx = cat_df.index
    num_df = df.select_dtypes(exclude = ['object'])
    ohe = OneHotEncoder(sparse = False)
    cat_df = ohe.fit_transform(cat_df)
    colnames = ohe.get_feature_names_out(ohe.feature_names_in_)
    cat_df = pd.DataFrame(cat_df, columns = colnames, index = cat_idx)
    #impute
    if imputer:
        num_df = imputer.fit_transform(num_df)
        colnames = imputer.feature_names_in_
        num_df = pd.DataFrame(num_df, columns = colnames)
        
    df = num_df.join(cat_df)

    colnames = df.columns
    return df, y, colnames 
    

In [5]:

df, df_y, colnames = preprocessing(train_df)

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(df, df_y, random_state = 12)
smoteenn = SMOTEENN()
sampled_data, sampled_data_y = smoteenn.fit_resample(xtrain, ytrain)

In [14]:
smoteen_df = pd.DataFrame(sampled_data, columns = colnames)
smoteen_target = pd.DataFrame(sampled_data_y)
smoteen_test = pd.DataFrame(xtest, columns = colnames)
smoteen_test_y = pd.DataFrame(ytest)
smoteen_df.to_csv('smoteen_df.csv', index = False)
smoteen_target.to_csv('smoteen_target.csv',index = False)
smoteen_test.to_csv('smoteen_test.csv',index = False)
smoteen_test_y.to_csv('smoteen_test_target.csv',index = False)

In [13]:
smoteen_test_y.head()

,TARGET
SK_ID_CURR,
320089.0,0.0
142995.0,0.0
378359.0,1.0
453298.0,0.0
246639.0,0.0


In [8]:
smoteen_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325580 entries, 0 to 325579
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   TARGET  325580 non-null  float64
dtypes: float64(1)
memory usage: 2.5 MB


In [9]:
smoteen_target['TARGET'].value_counts()/smoteen_target.shape[0]

1.0    0.586934
0.0    0.413066
Name: TARGET, dtype: float64